In [ ]:
(version)

In [ ]:
#lang iracket/lang #:require bhdl/splicing #:reader bhdl/splicing

In [ ]:
(list (list 1 2 3) ..)

In [ ]:
(putenv "BHDL_LIBRARY_PATH" "~/bhdl/bhdl-footprints")
(getenv "BHDL_LIBRARY_PATH")

In [ ]:
(require racket/list
         racket/format
         (prefix-in pict: pict))

In [ ]:
(require bhdl)

In [ ]:
(require "fitboard-lib.rkt")

In [ ]:
(define global
  (make-circuit
   #:external-pins (GND 3V3 5V VCC USB5V)))

# The keyboard matrix

In [ ]:
(define-key-matrix matrix
  ([(esc 1.5)    1 2 3 4    5       -         -        6 7 8  9  0   (backspace 1.5)]
   [(tab 1.5)    Q  W  E  R     T        -         -        Y  U  I   O   P    (\\ 1.5)]
   [(caps 1.5)   A  S  D  F     G        (lmid1 1.5) (rmid1 1.5)        H  J  K   L   \;  (enter 1.5)]
   [(lshift 1.5) Z  X  C  V     B        (lmid2 1.5) (rmid2 1.5)        N  M  \, \. \/  (rshift 1.5)]
   [lctrl lfn lsuper lalt lTBD (lspace 2.75) - - (rspace 2.75) rTBD ralt rsuper rfn rctrl]))

In [ ]:
kQ

In [ ]:
(show-layout kQ)

In [ ]:
;; FIXME the size is bigger. This might create problems
(show-layout klspace)

In [ ]:
(define matrix-module
  (make-circuit
   ;; FIXME these should be row[5] col[14], or get from matrix
   #:external-pins (row1 row2 row3 row4 row5
                         col1 col2 col3 col4 col5 col6 col7
                         col8 col9 col10 col11 col12 col13 col14)
   #:layout (make-fitboard-layout matrix)
   ;; connections
   #:connect (for/list ([x (range 5)])
               (filter-not
                void?
                (for/list ([y (range 14)])
                  (let ([key (keyboard-xy matrix x y)]
                        ;; FIXME this is ugly
                        [xname (string->symbol (~a "row" (add1 x)))]
                        [yname (string->symbol (~a "col" (add1 y)))])
                    (when key
                      (*-
                       (pin-ref self yname)
                       key
                       (pin-ref self xname)
                       ))))))))

In [ ]:
(show-layout matrix-module)

# MCU module

In [ ]:
;; FIXME I actually should not combine two atoms into one, because 
(define (icsp-header)
   (make-circuit #:vars ([h (PinHeader2 3)]
                         )
                   #:external-pins (MOSI SCK MISO RESET VCC GND)
                   #:connect (*= (self (MOSI SCK RESET))
                                 (h [1 2 3 4 5 6]))
                   #:layout h))

In [ ]:
(show-layout (icsp-header))

In [ ]:
;; Now using the Atmega32u4 from the arduino board
(define mcu-module 
  (make-circuit 
 #:vars ([mcu (ATmega32U4)]
         [usb (USB-C-16)]
         [icsp (icsp-header)])
 #:external-pins (
               ;; TODO how to abstract over pin names
               row1 row2 row3 row4 row5
               col1 col2 col3 col4 col5 col6 col7
               col8 col9 col10 col11 col12 col13 col14)
 #:layout (vc-append 50 (rotate usb pi) mcu icsp)
 ;; define what are the row and col scan pins
 #:connect (*= (self [row1 row2 row3 row4 row5])
                 (mcu [PB0 PB4 PB5 PB6 PB7]))
 #:connect (*= (self [col1 col2 col3 col4 col5 col6 col7
                             col8 col9 col10 col11 col12 col13 col14])
                 (mcu [PD0 PD1 PD2 PD3 PD4 PD5 PD6 PD7 PF0 PF1 PF4 PF5 PF6 PF7]))
   ;; use a Pin for blinking LED
   #:connect (*- mcu.PC6 (R '1k) (LED0603 'red) global.GND)
                    
 ;; connect the ICSP header
 #:connect (*= (mcu (MOSI SCLK MISO RESET VCC GND))
               (icsp (MOSI SCK MISO RESET VCC GND)))
 #:connect
 (list (*- mcu.VCC mcu.UVCC mcu.AVCC global.VCC)
         (*- mcu.GND mcu.UGND global.GND)
         ;; reset circuit and switch
         (*- mcu.RESET (*< (Diode)
                           (R '10k)) global.5V)
         (*- mcu.RESET (Switch) global.GND)
       ;; capacitors
       (*- mcu.UCAP (C '1uf) global.GND)
       (*- global.VCC (*< (C '1uf)
                          (C '100nf)
                          (C '100nf)
                          (C '100nf)
                          (C '100nf))
           global.GND)
         ;; crystal
         (*- mcu.XTAL1 (Crystal-2 '16Mhz) mcu.XTAL2)
         (*- mcu.XTAL1 (C '22pf) global.GND)
         (*- mcu.XTAL2 (C '22pf) global.GND)
       ;; USB
       (*- mcu.D+ (R 22) usb.D+1)
       (*- mcu.D- (R 22) usb.D-1)
       ;; just use USB for supplying power
       (*- mcu.VBUS usb.VBUS global.VCC)
       (*- mcu.UGND usb.GND global.GND)
       )))

In [ ]:
(show-layout mcu-module)

# Integrate into One board

In [ ]:
;; connect MCU with matrix
(define fitboard
  (make-circuit
   ;; CAUTION just to declare the pict
   #:layout (ct-superimpose matrix-module mcu-module)
   #:connect (list matrix-module mcu-module
                   ;; connect matrix module with gd32 module
                   ;; TODO *= connects corresponding pins with same name by default
                   (*= (matrix-module [row1 row2 row3 row4 row5])
                       (mcu-module [row1 row2 row3 row4 row5]))
                   (*= (matrix-module [col1 col2 col3 col4 col5 col6 col7
                                            col8 col9 col10 col11 col12 col13 col14])
                       (mcu-module [col1 col2 col3 col4 col5 col6 col7
                                          col8 col9 col10 col11 col12 col13 col14]))
                   )))


In [ ]:
(show-layout fitboard)

In [ ]:
(parameterize ([current-directory "./out/unplaced"])
    (circuit-export fitboard
                    #:auto-place #f
                    #:formats '(kicad pdf dsn)))

In [ ]:
#;
(parameterize ([current-directory "./out-placed"])
    (circuit-export fitboard
                    #:auto-place #t
                    #:formats '(kicad pdf dsn)))

In [ ]:
;; This is slow. Disabling
#;
(parameterize ([current-directory "./out/placed-5"]
               ;; add more padding
               [padding-general 5])
    (circuit-export fitboard
                    #:auto-place #t
                    #:formats '(kicad pdf dsn)))

# Modularized Design

Separate boards using Link!

This is fantastic. I want even further:
- save the placement results internally
- Use the placed module
- route the module and store the routing result

Now probably something easier:
- just mirror the output header
- put the same headers in the outer board

I can have more benefit doing this:
- the left and right half can be independently manufactured!

In [ ]:
;; actually I want to define the connector module
(define the-link
  (make-circuit
   #:external-pins (row1 row2 row3 row4 row5
                          col1 col2 col3 col4 col5 col6 col7
                          col8 col9 col10 col11 col12 col13 col14)
   #:layout (hb-append 50
                       col-left 
                       (rotate row1 (/ pi 2))
                       (rotate row2 (/ pi 2))
                       col-right)
   #:vars ([row1 (PinHeader 5)]
           [row2 (PinHeader 5)]
           [col-left (PinHeader 7)]
           [col-right (PinHeader 7)])
   #:connect (list (*= (self [row1 row2 row3 row4 row5])
                       (row1 [1 2 3 4 5])
                       (row2 [1 2 3 4 5]))
                   (*= (self [col1 col2 col3 col4 col5 col6 col7
                                            ])
                       (col-left [1 2 3 4 5 6 7]))
                   (*= (self [col8 col9 col10 col11 col12 col13 col14])
                       (col-right [1 2 3 4 5 6 7])))))

In [ ]:
(show-layout the-link)

In [ ]:
;; connect MCU
(define mcu-board
  (make-circuit 
   #:layout (cb-superimpose mcu-module the-link)
   #:connect (*= (the-link [row1 row2 row3 row4 row5
                          col1 col2 col3 col4 col5 col6 col7
                          col8 col9 col10 col11 col12 col13 col14])
                 (mcu-module [row1 row2 row3 row4 row5
                          col1 col2 col3 col4 col5 col6 col7
                          col8 col9 col10 col11 col12 col13 col14]))))

In [ ]:
(show-layout mcu-board)

In [ ]:
(parameterize ([current-directory "./out/mcu-board"]
               [padding-general 2])
    (circuit-export mcu-board
                    #:auto-place #t
                    #:formats '(kicad pdf dsn)))

In [ ]:
(mounting-hole 3)

In [ ]:
(show-layout (mounting-hole 3))

In [ ]:
;; The matrix
(define matrix-board
  (make-circuit
   #:external-pins (dummy1 dummy2 dummy3 dummy4)
   #:layout (cc-superimpose matrix-module
                            ;; FIXME simplify the layout
                             (vc-append the-link (pict:ghost (pict:rectangle 1 550)))
                             (vc-append (hc-append 1200 hole1 hole2)
                                        (pict:ghost (pict:rectangle 1 600)))
                             (vc-append (pict:ghost (pict:rectangle 1 500))
                                        (hc-append 1000 hole3 hole4)))
   #:vars ([hole1 (mounting-hole 3)]
           [hole2 (mounting-hole 3)]
           [hole3 (mounting-hole 3)]
           [hole4 (mounting-hole 3)])
   #:connect (list (*= (self [dummy1 dummy2 dummy3 dummy4])
                       ([hole1.1 hole2.1 hole3.1 hole4.1]))
                   (*= (matrix-module [row1 row2 row3 row4 row5])
                       (the-link [row1 row2 row3 row4 row5]))
                   (*= (matrix-module [col1 col2 col3 col4 col5 col6 col7
                                            col8 col9 col10 col11 col12 col13 col14])
                       (the-link [col1 col2 col3 col4 col5 col6 col7
                                          col8 col9 col10 col11 col12 col13 col14])))))

In [ ]:
(show-layout matrix-board)

In [ ]:
(mounting-hole 3)

In [ ]:
(parameterize ([current-directory "./out/matrix-board"]
               [padding-general 2])
    (circuit-export matrix-board
                    #:auto-place #t
                    #:formats '(kicad pdf dsn)))

# Other Notes

## Debug the deterministic

In [ ]:
(define (kailh-socket? x)
  (string=? "kailh-socket" (IC-name (ICAtom-ic x))))

In [ ]:
(define (kX->X k)
  (findf kailh-socket?
          (collect-all-atoms k)))

In [ ]:
;; this is deterministic
(define Hatom=>index (for/hash ([atom (collect-all-atoms fitboard)]
                                [i (in-naturals 1)])
          (values atom i)))

In [ ]:
;; Expect 28
(hash-ref Hatom=>index (kX->X kQ))

In [ ]:
;; Expect 94
(hash-ref Hatom=>index (kX->X kN))

In [ ]:
;; this is different!
;; UPDATE ... not very stable
(define Hatom=>index2 (for/hash ([atom (collect-all-atoms-random fitboard)]
           [i (in-naturals 1)])
          (values atom i)))

In [ ]:
(hash-ref Hatom=>index2 (kX->X kQ))

In [ ]:
(hash-ref Hatom=>index2 (kX->X kN))

# Generating the BOM

- I should have an identifier of which version this board is, so that the deterministic order won't change. Probably associate with git version?